In [6]:
from google import genai
import base64
import os
from google.genai import types
import mimetypes
import cv2
import tempfile


In [7]:
PROJECT_ID = "videoanalysis-model" 
REGION = "us-central1" 
generate_content_config = types.GenerateContentConfig(
    response_mime_type="application/json",
    temperature=0.0,
    max_output_tokens=8100,
    top_p=0.95,
    candidate_count=1,
)

In [8]:
client = genai.Client(vertexai=True, project=PROJECT_ID, location=REGION)

In [9]:

def extract_base64_frames_with_mime(video_path):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = int(total_frames / fps)

    image_parts = []

    for second in range(duration):
        cap.set(cv2.CAP_PROP_POS_MSEC, second * 1000)
        success, frame = cap.read()
        if not success:
            break

        # Save frame temporarily to get MIME type
        with tempfile.NamedTemporaryFile(suffix=".jpg", delete=False) as tmp_file:
            temp_filename = tmp_file.name
            cv2.imwrite(temp_filename, frame)

        # Read and encode the image in base64
        with open(temp_filename, "rb") as f:
            encoded_image = base64.b64encode(f.read()).decode('utf-8')

        # Get MIME type
        mime_type, _ = mimetypes.guess_type(temp_filename)

        # Create parts
        image_part = types.Part.from_bytes(
            mime_type=mime_type,
            data=base64.b64decode(encoded_image)
        )

        image_parts.append(image_part)

        # Remove the temporary file
        os.remove(temp_filename)

    cap.release()
    return image_parts


In [10]:
with open('./test_prompt.txt', 'r') as file:
    selected_prompt = file.read()
prompt_part = types.Part.from_text(text=selected_prompt)

In [ ]:
# process one video at a time

video_path = "./0_true_positive/trim_videos/2025-05-04_13-56-03-860029_clip_990_part_002.mp4"
image_parts = extract_base64_frames_with_mime(video_path)

image_parts.append(prompt_part)
# Create content
contents = [
    types.Content(
        role="user",
        parts= image_parts ,
    ),
]

response = client.models.generate_content(
                model="projects/1078720353462/locations/us-central1/endpoints/3754397901762068480",
                contents=contents,
                config=generate_content_config,
            )

print(response.text)

In [11]:
# process all videos inside the folder_path

folder_path = "./test/t"
output_file = "./0_true_positive/result.txt"
# Video file extensions to look for
video_extensions = ('.mp4', '.avi', '.mov', '.mkv', '.flv', '.wmv', '.webm')

for file in os.listdir(folder_path):
    if file.lower().endswith(video_extensions):
        video_path = os.path.join(folder_path, file)
        image_parts = extract_base64_frames_with_mime(video_path)
        image_parts.append(prompt_part)
        contents = [
            types.Content(
                role="user",
                parts= image_parts ,
            ),
        ]
        response = client.models.generate_content(
                model="projects/1078720353462/locations/us-central1/endpoints/3754397901762068480",
                contents=contents,
                config=generate_content_config,
            )
        with open(output_file, 'a') as f:
            f.write(video_path + " -- \n" + response.text + '\n \n')
        